### Ⅰ. 라이브러리

In [3]:
import numpy as np
import pandas as pd

import os
import mido

from mido import MidiFile, MidiTrack, MetaMessage, Message




### Ⅱ. 데이터 추출

#### [1]. MetaMessage
- tracks[0] : 기본 정보
    + MetaMessage('track_name', name="He's A Pirate", time=0),
    + MetaMessage('text', text='By Written by Klaus Badelt', time=0),
    + MetaMessage('copyright', text='Arrangement © 2004 Cerullean Pictures ', time=0),
    + MetaMessage('copyright', text='Original Score © Walt Disney Pictures', time=0),
    + MetaMessage('text', text='Generated by NoteWorthy Composer', time=0),
    + MetaMessage('set_tempo', tempo=300000, time=0), ✔️
    + MetaMessage('key_signature', key='F', time=0), ✔️
    + MetaMessage('time_signature', numerator=6, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0), ✔️
    + MetaMessage('end_of_track', time=0)]) ✔️
    ----
    + MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0), ✔️
    + MetaMessage('set_tempo', tempo=833333, time=0), ✔️
    + MetaMessage('set_tempo', tempo=857143, time=31800),
    + MetaMessage('end_of_track', time=0)]) ✔️
    ----
    (1). midi_port <br/>
    (2). set_tempo <br/>
    (3). smpte_offset <br/>
    (4). time_signature <br/>
    (5). key_signature <br/>
- tracks[1] : 
    + MetaMessage('midi_port', port=0, time=0),
    + MetaMessage('track_name', name='Right Hand', time=0),
    -----
    + MetaMessage('text', text='RH B', time=0),
    

#### [2]. Message
- type
    - note_on : 음표 시작
    - control_change : 컨트롤러 변경 메시지 ex. 페달 볼륨 조절기 움직임
    - program_change : 프로그램 변경 메시지 ex. 다양한 소리 또는 패치

#### 1. pitch (음정) : [  -8192 to 8192 ]
- 음의 높낮이
- 주파수 표현

#### 2. note (음표) : [  0 to 127 ]
- msg.type = 'note_on' / 'note_off'
    - 소리의 강도 / 세기 ( velocity )
    - 지속 시간 ( time, 단위 : tick )

#### 3. control ( 컨트롤러 번호 ) [ 0 to 127 ]
- https://www.midi.org/specifications/midi-reference-tables/midi-1-0-control-change-messages-data-bytes-2
----
- control=1  ( Modulation Wheel ( 음향 변화 )) ( 0 : 왼쪽, 63 : 중앙 127 : 오른쪽 )
- control=7  ( Main Volume 조절 )
- control=10 ( Pan, 좌우 스테레오 위치 조절 )
- control=64 ( Damper Pedal on/off (Sustain) ) <= 63 Off 64<= On

#### 4. program ( 특정 악기 / 사운드 패치 ) [ 0 to 127 ]
- program_change : 이벤트를 통해, 악기를 변경
- 피아노, 비트 등의 특정한 사운드 지칭
----
- program=0 : 피아노
- program=25 : 악기 관련 사운드
- program=40 : 비트 드럼

In [4]:
file_name = "piano_pirate.mid"
file_path = os.path.join("midi_data", file_name)

mid = MidiFile(file_path)

In [5]:
mid

MidiFile(type=1, ticks_per_beat=192, tracks=[
  MidiTrack([
    MetaMessage('track_name', name="He's A Pirate", time=0),
    MetaMessage('text', text='By Written by Klaus Badelt', time=0),
    MetaMessage('copyright', text='Arrangement © 2004 Cerullean Pictures ', time=0),
    MetaMessage('copyright', text='Original Score © Walt Disney Pictures', time=0),
    MetaMessage('text', text='Generated by NoteWorthy Composer', time=0),
    MetaMessage('set_tempo', tempo=300000, time=0),
    MetaMessage('key_signature', key='F', time=0),
    MetaMessage('time_signature', numerator=6, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('end_of_track', time=0)]),
  MidiTrack([
    MetaMessage('midi_port', port=0, time=0),
    MetaMessage('track_name', name='Right Hand', time=0),
    Message('control_change', channel=0, control=64, value=0, time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('control_change', channel=0, control=

In [16]:
def distribute_msg(mid_track):
    track_dict = {}
    
    track_dict['MetaMessage'] = []
    track_dict['Message'] = []
    track_dict['ETC'] = []
    
    for msg in mid_track:
        
        if isinstance(msg, MetaMessage):
            track_dict['MetaMessage'].append(msg)
            
        elif isinstance(msg, Message):
            track_dict['Message'].append(msg)
            
        else:
            track_dict['ETC'].append(msg)
            
    return track_dict

In [14]:
mid1.tracks[0]

MidiTrack([
  MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
  MetaMessage('set_tempo', tempo=833333, time=0),
  MetaMessage('set_tempo', tempo=857143, time=31800),
  MetaMessage('set_tempo', tempo=882353, time=120),
  MetaMessage('set_tempo', tempo=909091, time=120),
  MetaMessage('set_tempo', tempo=937500, time=120),
  MetaMessage('set_tempo', tempo=967742, time=120),
  MetaMessage('set_tempo', tempo=1000000, time=120),
  MetaMessage('set_tempo', tempo=1200000, time=120),
  MetaMessage('set_tempo', tempo=1224490, time=120),
  MetaMessage('set_tempo', tempo=833333, time=120),
  MetaMessage('end_of_track', time=0)])

#### 4. time // 이것을 어떻게 할 지 좀 더 고민
- tick : mid.ticks_per_beat
    - 1박자의 tick 수를 의미
- tick_interval 을 의미
- time = 0 ( interval = 0 )
    - 이전 이벤트와 동시에 발생
    - 동일한 시간에 기록

In [37]:
mid.tracks[0]

MidiTrack([
  MetaMessage('track_name', name="He's A Pirate", time=0),
  MetaMessage('text', text='By Written by Klaus Badelt', time=0),
  MetaMessage('copyright', text='Arrangement © 2004 Cerullean Pictures ', time=0),
  MetaMessage('copyright', text='Original Score © Walt Disney Pictures', time=0),
  MetaMessage('text', text='Generated by NoteWorthy Composer', time=0),
  MetaMessage('set_tempo', tempo=300000, time=0),
  MetaMessage('key_signature', key='F', time=0),
  MetaMessage('time_signature', numerator=6, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
  MetaMessage('end_of_track', time=0)])

In [42]:

mido.tick2second(mid.tracks[1][12].time,mid.ticks_per_beat, 300000)

0.049999999999999996

In [63]:
def initialize_meta_dict():
    df_meta = {}
    
    ############################################
    ##### default setting
    ############################################
    
    df_meta['midi_port'] = 0
    df_meta['set_tempo'] =  500000
    
    df_meta['smpte_offset'] = {}
    df_meta['smpte_offset']['frame_rate'] = 24
    df_meta['smpte_offset']['hours'] = 0
    df_meta['smpte_offset']['minutes'] = 0
    df_meta['smpte_offset']['seconds'] = 0
    df_meta['smpte_offset']['frames'] = 0
    df_meta['smpte_offset']['subframes'] = 0
    
    df_meta['time_signature'] = {}
    df_meta['time_signature']['numerator'] = 4
    df_meta['time_signature']['denominator'] = 4
    df_meta['time_signature']['clocks_per_click'] = 24
    df_meta['time_signature']['notated_32nd_notes_per_beat'] = 8
    
    df_meta['key_signature'] = "C"
    
    return df_meta

a = initialize_meta_dict()
a['key_signature'].replace(a['key_signature'], "AA")

'AA'

In [65]:
for idx, track in enumerate(mid.tracks):
    print(f"Track {idx + 1}: {track.name}")
    
    track_dict = distribute_msg(track)
    
    ###########################################################
    ########### 'MetaMessage'
    ###########################################################
    
    df_meta = initialize_meta_dict()
    
    print(df_meta)
    print("====================")
    if len(track_dict['MetaMessage'])>0:

        
        track_mtmsg = track_dict['MetaMessage']
        
        for mtmsg in track_dict['MetaMessage']:
            if mtmsg.is_meta:
                                
                
                if mtmsg.type == 'midi_port':
                    df_meta[mtmsg.type] = mtmsg.port
                    
                elif mtmsg.type == 'set_tempo':
                    df_meta[mtmsg.type] = mtmsg.tempo
                    
                elif mtmsg.type == 'smpte_offset':
                    df_meta[mtmsg.type]['frame_rate'] = mtmsg.frame_rate
                    df_meta[mtmsg.type]['hours'] = mtmsg.hours
                    df_meta[mtmsg.type]['minutes'] = mtmsg.minutes
                    df_meta[mtmsg.type]['seconds'] = mtmsg.seconds
                    df_meta[mtmsg.type]['frames'] = mtmsg.frames
                    df_meta[mtmsg.type]['subframes'] = mtmsg.subframes
                    
                elif mtmsg.type == 'time_signature':
                    df_meta[mtmsg.type]['numerator'] = mtmsg.numerator
                    df_meta[mtmsg.type]['denominator'] = mtmsg.denominator
                    df_meta[mtmsg.type]['clocks_per_click'] = mtmsg.clocks_per_click
                    df_meta[mtmsg.type]['notated_32nd_notes_per_beat'] = mtmsg.notated_32nd_notes_per_beat
                    
                elif mtmsg.type == 'key_signature':
                    df_meta[mtmsg.type] = mtmsg.key
              
            # 템포가 여러개일 경우
            # 코드 정리
            # 비교 알고리즘
        
    

Track 1: He's A Pirate
{'midi_port': 0, 'set_tempo': 500000, 'smpte_offset': {'frame_rate': 24, 'hours': 0, 'minutes': 0, 'seconds': 0, 'frames': 0, 'subframes': 0}, 'time_signature': {'numerator': 4, 'denominator': 4, 'clocks_per_click': 24, 'notated_32nd_notes_per_beat': 8}, 'key_signature': 'C'}
{'midi_port': 0, 'set_tempo': 300000, 'smpte_offset': {'frame_rate': 24, 'hours': 0, 'minutes': 0, 'seconds': 0, 'frames': 0, 'subframes': 0}, 'time_signature': {'numerator': 6, 'denominator': 8, 'clocks_per_click': 24, 'notated_32nd_notes_per_beat': 8}, 'key_signature': 'F'}


In [3]:
file_name1 = "Prelude1.mid"
file_path1 = os.path.join("midi_data", file_name1)

mid1 = MidiFile(file_path1)

In [230]:
def initialize_note_ticks():
    tick_type = []
    tick_note = []
    tick_chan = []
    tick_velo = []
    
    return tick_type, tick_note, tick_chan, tick_velo

def initialize_ctl_ticks():
    tick_chan = []
    tick_ctl  = []
    tick_val  = []
    
    return tick_chan, tick_ctl, tick_val

def initialize_prog_ticks():
    tick_chan = []
    tick_prog  = []
    
    return tick_chan, tick_prog

In [15]:
for idx, track in enumerate(mid.tracks):
    print(f"Track {idx + 1}: {track.name}")
    
    track_dict = distribute_msg(track)
    
    ###########################################################
    ########### 'MetaMessage'
    ###########################################################
    
    if len(track_dict['Message'])==0:
        
        continue;
    
    
    ###########################################################
    ########### 'Message'
    ###########################################################

    track_msg = track_dict['Message']
    
    df_note_col = {'tick', 'count', 'msg_type', 'msg_note', 'msg_velocity', 'msg_channel'}
    df_note = pd.DataFrame(columns = df_note_col)
    
    df_control_col = {'tick', 'count','channel', 'control', 'value'}
    df_ctl = pd.DataFrame(columns = df_control_col)
    
    df_program_col = {'tick', 'count', 'channel', 'program'}
    df_prog = pd.DataFrame(columns = df_program_col)
    
    
    note_idx = 0
    note_cur_time = 0
    note_start = True
    note_tick_type, note_tick_note, note_tick_chan, note_tick_velo = initialize_note_ticks()
    
    ctl_idx = 0
    ctl_cur_time = 0
    ctl_start = True
    ctl_tick_chan, ctl_tick_ctl, ctl_tick_val = initialize_ctl_ticks()
    
    prog_idx = 0
    prog_cur_time = 0
    prog_start = True
    prog_tick_chan, prog_tick_prog = initialize_prog_ticks()
    
    
    for msg in track_msg:
        
        # note off : note_on and velocity == 0
        if msg.type == 'note_on' and msg.velocity == 0:
            msg_typ = 'note_off'
        else:
            msg_typ = msg.type
            
        
        if msg.type == 'note_on' or msg.type == 'note_off':
            if msg.time > 0 :
                
                if note_start == True:
                    note_cur_time = msg.time + note_cur_time
                    note_start = False;
            
                else:
                    note_cur_time = msg.time + note_cur_time
                    note_idx+=1
            
                    df_note.loc[note_idx, "tick"] = note_cur_time
                    df_note.loc[note_idx, "count"] = len(note_tick_note)
                    df_note.loc[note_idx, "msg_type"] = note_tick_type
                    df_note.loc[note_idx, "msg_note"] = note_tick_note
                    df_note.loc[note_idx, "msg_velocity"] = note_tick_velo
                    df_note.loc[note_idx, "msg_channel"] = note_tick_chan
            
                    note_tick_type, note_tick_note, note_tick_chan, note_tick_velo = initialize_note_ticks()
            
            note_tick_type.append(msg_typ)
            note_tick_note.append(msg.note)
            note_tick_chan.append(msg.channel)
            note_tick_velo.append(msg.velocity)
            
        if msg.type == 'control_change':
            
            if msg.time > 0:
                
                if ctl_start == True:
                    ctl_cur_time = msg.time + ctl_cur_time
                    ctl_start = False;
                    
                else:
                    ctl_cur_time = msg.time + ctl_cur_time
                    ctl_idx +=1
                
                    df_ctl.loc[ctl_idx, "tick"] = ctl_cur_time
                    df_ctl.loc[ctl_idx, "count"] = len(ctl_tick_ctl)
                    df_ctl.loc[ctl_idx, "channel"] = ctl_tick_chan
                    df_ctl.loc[ctl_idx, "control"] = ctl_tick_ctl
                    df_ctl.loc[ctl_idx, "value"] = ctl_tick_val
                
                    ctl_tick_chan, ctl_tick_ctl, ctl_tick_val = initialize_ctl_ticks()
        
            ctl_tick_chan.append(msg.channel)
            ctl_tick_ctl.append(msg.control)
            ctl_tick_val.append(msg.value)
            
        if msg.type == 'program_change':
            
            if msg.time > 0:
                if prog_start == True:
                    prog_cur_time = msg.time + prog_cur_time
                    prog_start = False;
                    
                else:
                    prog_cur_time = msg.time + prog_cur_time
                    prog_idx +=1
                
                    df_prog.loc[prog_idx, "tick"] = prog_cur_time
                    df_prog.loc[prog_idx, "count"] = len(prog_tick_prog)
                    df_prog.loc[prog_idx, "channel"] = prog_tick_chan
                    df_prog.loc[prog_idx, "program"] = prog_tick_prog
                
                    prog_tick_chan, prog_tick_prog = initialize_prog_ticks()
        
            prog_tick_chan.append(msg.channel)
            prog_tick_prog.append(msg.program)
         
        #############################################################
        ############ 마지막 과정
        #############################################################
    if len(note_tick_note) > 0:
        note_cur_time = msg.time + note_cur_time
        note_idx+=1
            
        df_note.loc[note_idx, "tick"] = note_cur_time
        df_note.loc[note_idx, "count"] = len(note_tick_note)
        df_note.loc[note_idx, "msg_type"] = note_tick_type
        df_note.loc[note_idx, "msg_note"] = note_tick_note
        df_note.loc[note_idx, "msg_velocity"] = note_tick_velo
        df_note.loc[note_idx, "msg_channel"] = note_tick_chan
            
    if len(ctl_tick_ctl):
        ctl_cur_time = msg.time + ctl_cur_time
        ctl_idx +=1
                
        df_ctl.loc[ctl_idx, "tick"] = ctl_cur_time
        df_ctl.loc[ctl_idx, "count"] = len(ctl_tick_ctl)
        df_ctl.loc[ctl_idx, "channel"] = ctl_tick_chan
        df_ctl.loc[ctl_idx, "control"] = ctl_tick_ctl
        df_ctl.loc[ctl_idx, "value"] = ctl_tick_val
            
    if len(prog_tick_prog):
        prog_cur_time = msg.time + prog_cur_time
        prog_idx +=1
                
        df_prog.loc[prog_idx, "tick"] = prog_cur_time
        df_prog.loc[prog_idx, "count"] = len(prog_tick_prog)
        df_prog.loc[prog_idx, "channel"] = prog_tick_chan
        df_prog.loc[prog_idx, "program"] = prog_tick_prog
            
        
    print(note_cur_time)
    display(df_note)
    print('---------------------')
    display(df_ctl)
    print('---------------------')
    display(df_prog)
    print("==========================")
                        


Track 1: He's A Pirate


NameError: name 'distribute_msg' is not defined

In [1]:
def extract_note(mid_msg):
    
    df_note_col = {'tick', 'count', 'msg_type', 'msg_note', 'msg_velocity', 'msg_channel'}
    df_note = pd.DataFrame(columns = df_note_col)
    
    note_idx = 0
    note_cur_time = 0
    note_start = True
    note_tick_type, note_tick_note, note_tick_chan, note_tick_velo = initialize_note_ticks()
    
    def initialize_note_ticks():
        tick_type = []
        tick_note = []
        tick_chan = []
        tick_velo = []
    
        return tick_type, tick_note, tick_chan, tick_velo
    
    def insert_note_data():
        note_idx+=1
            
        df_note.loc[note_idx, "tick"] = note_cur_time
        df_note.loc[note_idx, "count"] = len(note_tick_note)
        df_note.loc[note_idx, "msg_type"] = note_tick_type
        df_note.loc[note_idx, "msg_note"] = note_tick_note
        df_note.loc[note_idx, "msg_velocity"] = note_tick_velo
        df_note.loc[note_idx, "msg_channel"] = note_tick_chan
            
        note_tick_type, note_tick_note, note_tick_chan, note_tick_velo = initialize_note_ticks()
    
    
    for msg in mid_msg:
        
        print(msg)
        if msg.type == 'note_on' and msg.velocity == 0:
            msg_typ = 'note_off'
        else:
            msg_typ = msg.type
            
        
        if msg.type == 'note_on':
            
            if msg.time > 0:
                
                if note_start == True:
                    note_start = False
                    
                else:
                    insert_note_data()
                
                note_cur_time = msg.time + note_cur_time
            
            
            note_tick_type.append(msg_typ)
            note_tick_note.append(msg.note)
            note_tick_chan.append(msg.channel)
            note_tick_velo.append(msg.velocity)
        
        else if msg.type == 'note_off':
            
        
    if len(note_tick_note) > 0:
        note_cur_time = msg.time + note_cur_time
        note_idx+=1
            
        df_note.loc[note_idx, "tick"] = note_cur_time
        df_note.loc[note_idx, "count"] = len(note_tick_note)
        df_note.loc[note_idx, "msg_type"] = note_tick_type
        df_note.loc[note_idx, "msg_note"] = note_tick_note
        df_note.loc[note_idx, "msg_velocity"] = note_tick_velo
        df_note.loc[note_idx, "msg_channel"] = note_tick_chan
    display(df_note)

NameError: name 'mid' is not defined

In [32]:
track_dict = {}
track_dict['MetaMessage'] = []
track_dict['MetaMessage'].append(345)

print(track_dict)

{'MetaMessage': [345]}


In [18]:
for idx, track in enumerate(mid.tracks):
    print(f"Track {i + 1}: {track.name}")
    
    for m

Track 1: He's A Pirate
Track 2: Right Hand
Track 3: Left Hand


In [9]:
mid.tracks[0]

MidiTrack([
  MetaMessage('track_name', name="He's A Pirate", time=0),
  MetaMessage('text', text='By Written by Klaus Badelt', time=0),
  MetaMessage('copyright', text='Arrangement © 2004 Cerullean Pictures ', time=0),
  MetaMessage('copyright', text='Original Score © Walt Disney Pictures', time=0),
  MetaMessage('text', text='Generated by NoteWorthy Composer', time=0),
  MetaMessage('set_tempo', tempo=300000, time=0),
  MetaMessage('key_signature', key='F', time=0),
  MetaMessage('time_signature', numerator=6, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
  MetaMessage('end_of_track', time=0)])

In [73]:
len(mid.tracks[1])

1285

In [14]:
file_name2 = "jesu1.mid"
file_path2 = os.path.join("midi_data", file_name2)

mid2 = MidiFile(file_path2)
print(mid2)

MidiFile(type=1, ticks_per_beat=120, tracks=[
  MidiTrack([
    MetaMessage('sequencer_specific', data=(0, 0, 65), time=0),
    MetaMessage('track_name', name='JESU        ', time=0),
    MetaMessage('key_signature', key='G', time=0),
    MetaMessage('midi_port', port=0, time=0),
    MetaMessage('time_signature', numerator=9, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('set_tempo', tempo=500000, time=38340),
    MetaMessage('end_of_track', time=0)]),
  MidiTrack([
    MetaMessage('track_name', name="Jesu, Joy of Man's", time=0),
    MetaMessage('instrument_name', name='Synths Strings 1', time=0),
    MetaMessage('midi_port', port=0, time=0),
    Message('control_change', channel=0, control=0, value=80, time=0),
    Message('control_change', channel=0, control=32, value=0, time=0),
    Message('program_change', channel=0, program=50, time=0),
    Message('note_on', channel=0, note=67, velocity=104, time=60),
    Message('note_on', channel=

In [15]:
len(mid2.tracks)

8

In [16]:
mid2.tracks[0]

MidiTrack([
  MetaMessage('sequencer_specific', data=(0, 0, 65), time=0),
  MetaMessage('track_name', name='JESU        ', time=0),
  MetaMessage('key_signature', key='G', time=0),
  MetaMessage('midi_port', port=0, time=0),
  MetaMessage('time_signature', numerator=9, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
  MetaMessage('set_tempo', tempo=500000, time=38340),
  MetaMessage('end_of_track', time=0)])

In [17]:
mid2.tracks[1]

MidiTrack([
  MetaMessage('track_name', name="Jesu, Joy of Man's", time=0),
  MetaMessage('instrument_name', name='Synths Strings 1', time=0),
  MetaMessage('midi_port', port=0, time=0),
  Message('control_change', channel=0, control=0, value=80, time=0),
  Message('control_change', channel=0, control=32, value=0, time=0),
  Message('program_change', channel=0, program=50, time=0),
  Message('note_on', channel=0, note=67, velocity=104, time=60),
  Message('note_on', channel=0, note=67, velocity=0, time=48),
  Message('note_on', channel=0, note=69, velocity=104, time=12),
  Message('note_on', channel=0, note=69, velocity=0, time=48),
  Message('note_on', channel=0, note=71, velocity=104, time=12),
  Message('note_on', channel=0, note=71, velocity=0, time=48),
  Message('note_on', channel=0, note=74, velocity=104, time=12),
  Message('note_on', channel=0, note=74, velocity=0, time=48),
  Message('note_on', channel=0, note=72, velocity=104, time=12),
  Message('note_on', channel=0, note=7

In [ ]:
// pending